# Beyond Sequential Planning

Your STRIPS planner works. It finds plans. But look at what it produces:

```
Plan:
1. Make coffee
2. Make toast
3. Read news
4. Get dressed
```

This is a **total-order plan** - a fixed sequence where each action must complete before the next starts.

## The Over-Commitment Problem

But wait. Do you really need to make coffee BEFORE toast? They're independent. You could:
- Do them in either order
- Do them in parallel
- Interleave them

The planner picked an arbitrary order and locked it in. **Over-commitment.**

This matters because:
- Parallel execution is wasted (coffee maker and toaster could run simultaneously)
- Can't adapt at execution time (if toaster breaks, coffee plan fails)
- Harder to coordinate multiple agents (they fight over the strict sequence)

**What if we only specify orderings that MATTER?**

In [1]:
from collections import defaultdict
from typing import Set, List, Tuple, Dict, Optional
from dataclasses import dataclass
import graphviz

print("Libraries loaded")

Libraries loaded


## Partial-Order Planning

Partial-order plans specify actions and constraints, not sequences.

**Total-order plan:**
```
A₁ → A₂ → A₃ → A₄
```
Everything in strict sequence.

**Partial-order plan:**
```
Start
  ↓
  A₁ → A₃
  A₂ ↗
  ↓
Goal
```
A₁ and A₂ have no ordering - either order works. A₃ must follow A₁. That's the ONLY constraint.

### Components

**Actions:** Set of actions to execute (including Start and Finish)

**Ordering constraints:** A ≺ B means A must complete before B starts
- Not a total order - many actions unordered relative to each other
- Only constraints that are necessary for correctness

**Causal links:** A —p→ B means:
- A achieves precondition p
- B needs p
- Nothing between A and B can delete p

The causal link is a **protected** relationship. Breaking it breaks the plan.

In [2]:
@dataclass
class Action:
    name: str
    preconditions: Set[str]
    add_effects: Set[str]
    delete_effects: Set[str]
    
    def __hash__(self):
        return hash(self.name)
    
    def __repr__(self):
        return self.name

@dataclass(frozen=True)
class CausalLink:
    producer: Action  # Action that achieves the condition
    consumer: Action  # Action that needs the condition
    condition: str    # The condition being protected
    
    def __repr__(self):
        return f"{self.producer} —{self.condition}→ {self.consumer}"

class PartialOrderPlan:
    def __init__(self):
        self.actions = set()
        self.orderings = set()  # (A, B) means A ≺ B
        self.causal_links = set()
        
        # Special start/finish actions
        self.start = Action('Start', set(), set(), set())
        self.finish = Action('Finish', set(), set(), set())
        self.actions.add(self.start)
        self.actions.add(self.finish)
    
    def add_action(self, action: Action):
        self.actions.add(action)
        # New actions must come after Start and before Finish
        self.orderings.add((self.start, action))
        self.orderings.add((action, self.finish))
    
    def add_ordering(self, before: Action, after: Action):
        """Add constraint: before ≺ after"""
        self.orderings.add((before, after))
    
    def add_causal_link(self, producer: Action, consumer: Action, condition: str):
        link = CausalLink(producer, consumer, condition)
        self.causal_links.add(link)
        # Causal link implies ordering
        self.add_ordering(producer, consumer)

# Example: Making breakfast
make_coffee = Action('MakeCoffee', {'HasBeans'}, {'HasCoffee'}, set())
make_toast = Action('MakeToast', {'HasBread'}, {'HasToast'}, set())
eat = Action('Eat', {'HasCoffee', 'HasToast'}, {'Fed'}, {'HasCoffee', 'HasToast'})

plan = PartialOrderPlan()
plan.start.add_effects = {'HasBeans', 'HasBread'}  # Initial state
plan.finish.preconditions = {'Fed'}  # Goal

plan.add_action(make_coffee)
plan.add_action(make_toast)
plan.add_action(eat)

# Causal links
plan.add_causal_link(plan.start, make_coffee, 'HasBeans')
plan.add_causal_link(plan.start, make_toast, 'HasBread')
plan.add_causal_link(make_coffee, eat, 'HasCoffee')
plan.add_causal_link(make_toast, eat, 'HasToast')
plan.add_causal_link(eat, plan.finish, 'Fed')

print("Actions:", [a.name for a in plan.actions])
print("\nCausal Links:")
for link in plan.causal_links:
    print(f"  {link}")
print("\nMakeCoffee and MakeToast have NO ordering constraint.")
print("They can execute in either order or in parallel.")

Actions: ['Eat', 'MakeCoffee', 'MakeToast', 'Start', 'Finish']

Causal Links:
  MakeCoffee —HasCoffee→ Eat
  Eat —Fed→ Finish
  Start —HasBeans→ MakeCoffee
  MakeToast —HasToast→ Eat
  Start —HasBread→ MakeToast

MakeCoffee and MakeToast have NO ordering constraint.
They can execute in either order or in parallel.


## Threats

A **threat** occurs when an action might break a causal link.

```
A —p→ B
     ↑
     C (deletes p)
```

Action C threatens the link A —p→ B if:
1. C deletes condition p
2. C could execute between A and B

If C runs after A adds p but before B uses it, B's precondition fails.

### Resolving Threats

Two ways to resolve:

**Promotion:** Force C after B
```
A —p→ B ≺ C
```
C can't threaten because it runs after B is done.

**Demotion:** Force C before A
```
C ≺ A —p→ B
```
C can't threaten because it runs before A produces p.

The planner must detect threats and add ordering constraints to resolve them.

In [3]:
def find_threats(plan: PartialOrderPlan) -> List[Tuple]:
    """Find actions that threaten causal links"""
    threats = []
    
    for link in plan.causal_links:
        for action in plan.actions:
            if action == link.producer or action == link.consumer:
                continue
            
            # Does this action delete the protected condition?
            if link.condition in action.delete_effects:
                # Could it execute between producer and consumer?
                if not is_ordered_before(plan, action, link.producer) and \
                   not is_ordered_before(plan, link.consumer, action):
                    threats.append((action, link))
    
    return threats

def is_ordered_before(plan: PartialOrderPlan, a: Action, b: Action) -> bool:
    """Check if a ≺ b (transitive closure)"""
    if (a, b) in plan.orderings:
        return True
    
    # Check transitive: a ≺ x ≺ b
    for (x, y) in plan.orderings:
        if x == a and is_ordered_before(plan, y, b):
            return True
    return False

# Example with a threat
cleanup = Action('Cleanup', set(), set(), {'HasCoffee', 'HasToast'})  # Deletes both
plan.add_action(cleanup)

threats = find_threats(plan)
if threats:
    print("Threats detected:")
    for action, link in threats:
        print(f"  {action} threatens {link}")
    print("\nResolution: Promote Cleanup after Eat")
    plan.add_ordering(eat, cleanup)
    print(f"Added constraint: {eat} ≺ {cleanup}")
else:
    print("No threats")

Threats detected:
  Cleanup threatens MakeCoffee —HasCoffee→ Eat
  Cleanup threatens MakeToast —HasToast→ Eat

Resolution: Promote Cleanup after Eat
Added constraint: Eat ≺ Cleanup


## The Least-Commitment Principle

Partial-order planning follows **least commitment:** don't make decisions until you have to.

When building a plan:
- Don't order actions unless there's a causal dependency
- Don't choose variable bindings unless constrained
- Don't resolve threats until they actually occur

**Why this works:**

Early commitment can lead to backtracking. If you arbitrarily order A before B, then later discover that ordering prevents a solution, you backtrack and try B before A.

With least commitment, you leave A and B unordered until something forces an ordering. Less backtracking, more flexibility.

**The cost:**

More complex plan representation. Must track orderings, detect threats, maintain constraints. Total-order planners are simpler.

Trade flexibility for complexity.

## The Sussman Anomaly

A famous example showing why sequential subgoal planning fails.

**Initial state:**
```
C
A   B
Table
```

**Goal:**
```
  A
  B
  C
Table
```

Two subgoals: On(A, B) and On(B, C).

**Try achieving On(A, B) first:**
1. Unstack C from A
2. Stack A on B
3. Now need On(B, C)
4. But A is on B - must unstack A first
5. Undid the first subgoal!

**Try achieving On(B, C) first:**
1. Stack B on C (can't - C has A on it!)
2. Must unstack C from A first
3. Then stack B on C
4. Now need On(A, B)
5. Stack A on B
6. Works!

The correct solution **interleaves** the subgoals:
```
Unstack(C, A)   ← for subgoal 1
Stack(B, C)     ← for subgoal 2  
Stack(A, B)     ← for subgoal 1
```

**The lesson:** Subgoals can interact. Achieving one may require temporarily violating or postponing another.

Partial-order planning handles this naturally - it doesn't commit to subgoal order. It discovers the interleaving by adding only necessary constraints.

In [4]:
# Sussman Anomaly actions
unstack_C_A = Action('Unstack(C,A)', 
                     {'On(C,A)', 'Clear(C)'}, 
                     {'Clear(A)', 'Holding(C)'}, 
                     {'On(C,A)', 'Clear(C)'})

putdown_C = Action('Putdown(C)', 
                   {'Holding(C)'}, 
                   {'On(C,Table)', 'Clear(C)'}, 
                   {'Holding(C)'})

stack_B_C = Action('Stack(B,C)', 
                   {'Holding(B)', 'Clear(C)'}, 
                   {'On(B,C)', 'Clear(B)'}, 
                   {'Holding(B)', 'Clear(C)'})

pickup_B = Action('Pickup(B)', 
                  {'On(B,Table)', 'Clear(B)'}, 
                  {'Holding(B)'}, 
                  {'On(B,Table)', 'Clear(B)'})

stack_A_B = Action('Stack(A,B)', 
                   {'Holding(A)', 'Clear(B)'}, 
                   {'On(A,B)'}, 
                   {'Holding(A)', 'Clear(B)'})

pickup_A = Action('Pickup(A)', 
                  {'On(A,Table)', 'Clear(A)'}, 
                  {'Holding(A)'}, 
                  {'On(A,Table)', 'Clear(A)'})

sussman_plan = PartialOrderPlan()
sussman_plan.start.add_effects = {'On(C,A)', 'On(A,Table)', 'On(B,Table)', 'Clear(C)', 'Clear(B)'}
sussman_plan.finish.preconditions = {'On(A,B)', 'On(B,C)'}

# The solution interleaves both subgoals
for action in [unstack_C_A, putdown_C, pickup_B, stack_B_C, pickup_A, stack_A_B]:
    sussman_plan.add_action(action)

# Key orderings (only the necessary ones)
sussman_plan.add_ordering(unstack_C_A, putdown_C)
sussman_plan.add_ordering(putdown_C, stack_B_C)
sussman_plan.add_ordering(pickup_B, stack_B_C)
sussman_plan.add_ordering(stack_B_C, stack_A_B)
sussman_plan.add_ordering(pickup_A, stack_A_B)
sussman_plan.add_ordering(unstack_C_A, pickup_A)

print("Sussman Anomaly Solution:")
print("1. Unstack(C,A) - clear A for later")
print("2. Putdown(C)")
print("3. Pickup(B)")
print("4. Stack(B,C) - achieve subgoal 2")
print("5. Pickup(A)")
print("6. Stack(A,B) - achieve subgoal 1")
print("\nNotice: Actions from both subgoals are interleaved.")

Sussman Anomaly Solution:
1. Unstack(C,A) - clear A for later
2. Putdown(C)
3. Pickup(B)
4. Stack(B,C) - achieve subgoal 2
5. Pickup(A)
6. Stack(A,B) - achieve subgoal 1

Notice: Actions from both subgoals are interleaved.


## But Humans Don't Think Bottom-Up

Partial-order planning is flexible. But it still searches through low-level actions.

When you plan a trip, you don't think:
```
Turn door handle
Pull door open  
Step through doorway
Turn toward car
Walk to car
...
```

You think:
```
Get to airport
  ↳ Drive to airport
      ↳ Get in car
      ↳ Navigate to airport
      ↳ Park
```

**Hierarchical decomposition.** Abstract tasks break down into concrete steps.

And you use domain knowledge: "To get to airport, I can drive OR take a cab OR take train." You're not discovering these options through search - you already know them.

Can planning work this way?

## Hierarchical Task Network (HTN) Planning

HTN planning starts with an abstract task and recursively decomposes it.

**Input:** Initial state + Abstract task to achieve

**Output:** Sequence of primitive actions that accomplish the task

### Key Components

**Primitive tasks:** Executable actions (like STRIPS actions)
- Drive(A, B)
- PickUp(x)
- SendEmail(msg)

**Compound tasks:** Abstract tasks requiring decomposition
- Travel(Home, Airport)
- PrepareReport()
- DeliverPackage(x)

**Methods:** Recipes for decomposing compound tasks
```
Method: Travel-by-car
  Task: Travel(A, B)
  Preconditions: HaveCar, KnowRoute(A, B)
  Subtasks: 
    GetInCar
    Drive(A, B)
    ParkCar
```

A compound task can have MULTIPLE methods (alternative ways to achieve it).

In [5]:
@dataclass
class Task:
    name: str
    is_primitive: bool
    
    def __repr__(self):
        return self.name

@dataclass  
class Method:
    name: str
    task: Task  # The compound task this decomposes
    preconditions: Set[str]
    subtasks: List[Task]  # Ordered list of subtasks
    
    def __repr__(self):
        return f"{self.name}: {self.task} → [{', '.join(str(t) for t in self.subtasks)}]"

# Primitive tasks
get_in_car = Task('GetInCar', True)
drive = Task('Drive(Home,Airport)', True)
park = Task('Park', True)
call_taxi = Task('CallTaxi', True)
wait_for_taxi = Task('WaitForTaxi', True)
ride_taxi = Task('RideTaxi(Home,Airport)', True)
pay_taxi = Task('PayTaxi', True)

# Compound task
travel = Task('Travel(Home,Airport)', False)

# Methods (alternative ways to travel)
drive_self = Method(
    'Drive-Self',
    travel,
    {'HaveCar', 'CanDrive'},
    [get_in_car, drive, park]
)

take_taxi = Method(
    'Take-Taxi',
    travel,
    {'HaveMoney'},
    [call_taxi, wait_for_taxi, ride_taxi, pay_taxi]
)

print("Task:", travel)
print("\nMethod 1:", drive_self)
print("Method 2:", take_taxi)
print("\nPlanner chooses method based on preconditions and current state.")

Task: Travel(Home,Airport)

Method 1: Drive-Self: Travel(Home,Airport) → [GetInCar, Drive(Home,Airport), Park]
Method 2: Take-Taxi: Travel(Home,Airport) → [CallTaxi, WaitForTaxi, RideTaxi(Home,Airport), PayTaxi]

Planner chooses method based on preconditions and current state.


## HTN Planning Algorithm

Basic structure:

```python
def htn_plan(state, tasks):
    if tasks is empty:
        return []  # Success
    
    task = tasks[0]
    remaining = tasks[1:]
    
    if task.is_primitive:
        if task.applicable(state):
            new_state = task.apply(state)
            plan = htn_plan(new_state, remaining)
            return [task] + plan
        else:
            return None  # Fail
    
    else:  # Compound task
        for method in methods_for(task):
            if method.preconditions_met(state):
                subtasks = method.subtasks
                plan = htn_plan(state, subtasks + remaining)
                if plan:
                    return plan
        
        return None  # No method worked
```

Recursively decompose compound tasks. Execute primitives. Backtrack if a method fails.

In [6]:
class HTNPlanner:
    def __init__(self, methods: List[Method], primitives: Dict[str, Action]):
        self.methods = methods
        self.primitives = primitives
    
    def plan(self, state: Set[str], tasks: List[Task]) -> Optional[List[Task]]:
        """HTN planning with backtracking"""
        if not tasks:
            return []  # Success
        
        task = tasks[0]
        remaining = tasks[1:]
        
        if task.is_primitive:
            # Execute primitive action
            action = self.primitives.get(task.name)
            if action and action.preconditions.issubset(state):
                new_state = state - action.delete_effects
                new_state = new_state | action.add_effects
                rest_plan = self.plan(new_state, remaining)
                if rest_plan is not None:
                    return [task] + rest_plan
            return None
        
        else:
            # Decompose compound task
            for method in self.methods:
                if method.task.name == task.name:
                    if method.preconditions.issubset(state):
                        # Try this decomposition
                        subtasks = method.subtasks + remaining
                        plan = self.plan(state, subtasks)
                        if plan is not None:
                            return plan
            return None

# Define primitive actions
primitives = {
    'GetInCar': Action('GetInCar', {'HaveCar'}, {'InCar'}, set()),
    'Drive(Home,Airport)': Action('Drive(Home,Airport)', {'InCar', 'CanDrive'}, 
                                   {'AtAirport'}, {'AtHome'}),
    'Park': Action('Park', {'InCar', 'AtAirport'}, {'Parked'}, {'InCar'}),
}

planner = HTNPlanner([drive_self], primitives)
initial_state = {'HaveCar', 'CanDrive', 'AtHome'}
goal_tasks = [travel]

plan = planner.plan(initial_state, goal_tasks)
if plan:
    print("Plan found:")
    for i, task in enumerate(plan, 1):
        print(f"  {i}. {task}")
    print("\nDecomposed Travel(Home,Airport) into primitive actions.")
else:
    print("No plan found")

Plan found:
  1. GetInCar
  2. Drive(Home,Airport)
  3. Park

Decomposed Travel(Home,Airport) into primitive actions.


## Classical vs HTN Planning

**Classical Planning:**
- Input: Initial state + Goal state
- Search space: All possible action sequences
- Knowledge: Action preconditions/effects only
- Can find novel solutions
- Slow on large problems

**HTN Planning:**
- Input: Initial state + Task to accomplish  
- Search space: Valid decompositions
- Knowledge: Methods encode expert strategies
- Limited to defined methods
- Fast when methods are well-designed

### When to Use Each

**Use classical planning when:**
- Domain is simple, actions are obvious
- You want to discover novel solutions
- Engineering methods is too costly

**Use HTN planning when:**
- Domain experts have clear strategies
- Problems are large and structured
- Speed matters more than novelty
- Task hierarchy matches problem structure

## Real-World Applications

**Video game AI:**
NPCs in games like F.E.A.R. use HTN planning. Designers write methods for combat tactics:
```
AttackEnemy
  ↳ If HasAmmo: ShootFromCover
  ↳ If NoAmmo: FindAmmo then ShootFromCover  
  ↳ If Outnumbered: Retreat then CallReinforcements
```

**Manufacturing:**
Factory planning decomposes "build product" into assembly operations:
```
AssembleWidget
  ↳ FetchParts
  ↳ JoinComponentA_B
  ↳ AttachCover
  ↳ QualityCheck
```
Partial-order constraints enable parallel machine operations.

**LLM agents:**
Modern LLM systems use "decomposition prompting":
```
To solve X:
  1. First do Y
  2. Then do Z
  3. Finally do W
```
This mirrors HTN structure. Formalizing it with HTN might improve reliability.

## What We Built

**Problem 1:** Total-order plans over-commit → **Solution:** Partial-order planning (specify only necessary constraints)

**Problem 2:** Low-level action search is slow → **Solution:** HTN planning (decompose abstract tasks using methods)

**Problem 3:** Subgoals interact (Sussman anomaly) → **Solution:** Least-commitment approach finds interleaved solutions

Two different directions:
- Partial-order adds **flexibility** to plan structure
- HTN adds **abstraction** through hierarchical decomposition

Both are actively used:
- Partial-order for parallel execution and coordination
- HTN for large problems with known decomposition strategies

The fundamental insight: **How you represent plans affects what you can solve and how efficiently.**

Sequential plans forced artificial constraints. Removing them opened new possibilities.